# Embryo Data Query

This notebook queries embryo-related data from multiple databases for a given prontuario/PIN.

## Tables Queried:
- **Clinisys Silver Layer** (clinisys_all.duckdb):
  - view_micromanipulacao
  - view_micromanipulacao_oocitos
  - view_tratamentos
  - view_congelamentos_embrioes
  - view_descongelamentos_embrioes
  - view_embrioes_congelados
- **Huntington Data Lake** (huntington_data_lake.duckdb):
  - gold.clinisys_embrioes
  - gold.embryoscope_embrioes
  - gold.embryoscope_clinisys_combined
  - gold.embryoscope_treatment (from embryoscope_embrioes)
  - silver.planilha_embriologia
  - gold.planilha_embryoscope_combined


In [1]:
import duckdb
import pandas as pd
import os
from datetime import datetime

# Set pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


## Database Connections


In [2]:
def get_clinisys_connection():
    """Get connection to clinisys_all database"""
    clinisys_db_path = os.path.join('..', '..', 'database', 'clinisys_all.duckdb')
    return duckdb.connect(clinisys_db_path, read_only=True)

def get_huntington_connection():
    """Get connection to huntington_data_lake database"""
    huntington_db_path = os.path.join('..', '..', 'database', 'huntington_data_lake.duckdb')
    return duckdb.connect(huntington_db_path, read_only=True)


## Query Functions - Clinisys Tables


In [3]:
def query_micromanipulacao(conn, prontuario):
    """Query view_micromanipulacao for a specific prontuario"""
    print(f"\n=== VIEW_MICROMANIPULACAO ===")
    query = f"""
    SELECT * FROM silver.view_micromanipulacao 
    WHERE prontuario = {prontuario}
    ORDER BY codigo_ficha DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_micromanipulacao")
    return df

def query_micromanipulacao_oocitos(conn, prontuario):
    """Query view_micromanipulacao_oocitos for a specific prontuario"""
    print(f"\n=== VIEW_MICROMANIPULACAO_OOCITOS ===")
    
    # First get codigo_ficha from view_micromanipulacao for this prontuario
    codigos_query = f"""
    SELECT codigo_ficha FROM silver.view_micromanipulacao 
    WHERE prontuario = {prontuario}
    """
    
    codigos_df = conn.execute(codigos_query).df()
    if not codigos_df.empty:
        codigo_list = codigos_df['codigo_ficha'].tolist()
        placeholders = ','.join(['?'] * len(codigo_list))
        
        oocitos_query = f"""
        SELECT * FROM silver.view_micromanipulacao_oocitos
        WHERE id_micromanipulacao IN ({placeholders})
        ORDER BY id_micromanipulacao, id
        """
        
        df = conn.execute(oocitos_query, codigo_list).df()
        print(f"Found {len(df)} records in view_micromanipulacao_oocitos")
    else:
        print("No codigo_ficha found for this prontuario in view_micromanipulacao.")
        df = pd.DataFrame()
    
    return df


In [4]:
def query_tratamentos(conn, prontuario):
    """Query view_tratamentos for a specific prontuario"""
    print(f"\n=== VIEW_TRATAMENTOS ===")
    query = f"""
    SELECT * FROM silver.view_tratamentos 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_tratamentos")
    return df

def query_congelamentos_embrioes(conn, prontuario):
    """Query view_congelamentos_embrioes for a specific prontuario"""
    print(f"\n=== VIEW_CONGELAMENTOS_EMBRIOES ===")
    query = f"""
    SELECT * FROM silver.view_congelamentos_embrioes 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_congelamentos_embrioes")
    return df

def query_descongelamentos_embrioes(conn, prontuario):
    """Query view_descongelamentos_embrioes for a specific prontuario"""
    print(f"\n=== VIEW_DESCONGELAMENTOS_EMBRIOES ===")
    query = f"""
    SELECT * FROM silver.view_descongelamentos_embrioes 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_descongelamentos_embrioes")
    return df

def query_embrioes_congelados(conn, prontuario):
    """Query view_embrioes_congelados for a specific prontuario"""
    print(f"\n=== VIEW_EMBRIOES_CONGELADOS ===")
    query = f"""
    SELECT * FROM silver.view_embrioes_congelados 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_embrioes_congelados")
    return df

def query_congelamentos_ovulos(conn, prontuario):
    """Query view_congelamentos_ovulos for a specific prontuario"""
    print(f"\n=== VIEW_CONGELAMENTOS_OVULOS ===")
    query = f"""
    SELECT * FROM silver.view_congelamentos_ovulos 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_congelamentos_ovulos")
    return df

def query_descongelamentos_ovulos(conn, prontuario):
    """Query view_descongelamentos_ovulos for a specific prontuario"""
    print(f"\n=== VIEW_DESCONGELAMENTOS_OVULOS ===")
    query = f"""
    SELECT * FROM silver.view_descongelamentos_ovulos 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_descongelamentos_ovulos")
    return df

def query_ovulos_congelados(conn, prontuario):
    """Query view_ovulos_congelados for a specific prontuario"""
    print(f"\n=== VIEW_OVULOS_CONGELADOS ===")
    query = f"""
    SELECT * FROM silver.view_ovulos_congelados 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_ovulos_congelados")
    return df


## Query Functions - Huntington Data Lake Tables


In [5]:
def query_clinisys_embrioes(conn, prontuario):
    """Query gold.clinisys_embrioes for a specific prontuario"""
    print(f"\n=== GOLD.CLINISYS_EMBRIOES ===")
    query = f"""
    SELECT * FROM gold.clinisys_embrioes 
    WHERE micro_prontuario = {prontuario}
    ORDER BY micro_Data_DL DESC, micro_codigo_ficha DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in gold.clinisys_embrioes")
    return df

def query_embryoscope_embrioes(conn, prontuario):
    """Query gold.embryoscope_embrioes for a specific prontuario"""
    print(f"\n=== GOLD.EMBRYOSCOPE_EMBRIOES ===")
    query = f"""
    SELECT * FROM gold.embryoscope_embrioes 
    WHERE prontuario = {prontuario}
    ORDER BY embryo_KIDDate DESC, treatment_TreatmentName DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in gold.embryoscope_embrioes")
    return df

def query_embryoscope_clinisys_combined(conn, prontuario):
    """Query gold.embryoscope_clinisys_combined for a specific prontuario"""
    print(f"\n=== GOLD.EMBRYOSCOPE_CLINISYS_COMBINED ===")
    query = f"""
    SELECT * FROM gold.embryoscope_clinisys_combined 
    WHERE micro_prontuario = {prontuario}
    ORDER BY micro_Data_DL DESC, micro_codigo_ficha DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in gold.embryoscope_clinisys_combined")
    return df


In [6]:
def query_planilha_embriologia(conn, prontuario):
    """Query silver.planilha_embriologia for a specific prontuario"""
    print(f"\n=== SILVER.PLANILHA_EMBRIOLOGIA ===")
    query = f"""
    SELECT * FROM silver.planilha_embriologia 
    WHERE PIN = {prontuario}
    ORDER BY "DATA DA FET" DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in silver.planilha_embriologia")
    return df

def query_planilha_embryoscope_combined(conn, prontuario):
    """Query gold.planilha_embryoscope_combined for a specific prontuario"""
    print(f"\n=== GOLD.PLANILHA_EMBRYOSCOPE_COMBINED ===")
    query = f"""
    SELECT * FROM gold.planilha_embryoscope_combined 
    WHERE "micro_prontuario" = {prontuario} OR "planilha_PIN" = {prontuario}
    ORDER BY "descong_em_DataTransferencia" DESC, "planilha_DATA_DA_FET" DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in gold.planilha_embryoscope_combined")
    return df


## Main Query Function


In [7]:
def query_embryo_data(prontuario):
    """
    Query all embryo-related data for a given prontuario/PIN.
    
    Args:
        prontuario (str or int): The patient prontuario number
        
    Returns:
        dict: Dictionary containing all DataFrames with results
    """
    print(f"\n{'='*80}")
    print(f"QUERYING EMBRYO DATA FOR PRONTUARIO: {prontuario}")
    print(f"Query executed at: {datetime.now()}")
    print(f"{'='*80}")
    
    # Connect to databases
    clinisys_con = get_clinisys_connection()
    huntington_con = get_huntington_connection()
    
    try:
        # Query clinisys tables
        df_micromanipulacao = query_micromanipulacao(clinisys_con, prontuario)
        df_oocitos = query_micromanipulacao_oocitos(clinisys_con, prontuario)
        df_tratamentos = query_tratamentos(clinisys_con, prontuario)
        df_congelamentos_embrioes = query_congelamentos_embrioes(clinisys_con, prontuario)
        df_descongelamentos_embrioes = query_descongelamentos_embrioes(clinisys_con, prontuario)
        df_embrioes_congelados = query_embrioes_congelados(clinisys_con, prontuario)
        df_congelamentos_ovulos = query_congelamentos_ovulos(clinisys_con, prontuario)
        df_descongelamentos_ovulos = query_descongelamentos_ovulos(clinisys_con, prontuario)
        df_ovulos_congelados = query_ovulos_congelados(clinisys_con, prontuario)
        
        # Query huntington data lake tables
        df_clinisys_embrioes = query_clinisys_embrioes(huntington_con, prontuario)
        df_embryoscope_embrioes = query_embryoscope_embrioes(huntington_con, prontuario)
        df_embryoscope_clinisys_combined = query_embryoscope_clinisys_combined(huntington_con, prontuario)
        # df_embryoscope_treatment = query_embryoscope_treatment(huntington_con, prontuario)
        df_planilha_embriologia = query_planilha_embriologia(huntington_con, prontuario)
        df_planilha_embryoscope_combined = query_planilha_embryoscope_combined(huntington_con, prontuario)
        
        # Summary
        print(f"\n{'='*80}")
        print(f"QUERY SUMMARY FOR PRONTUARIO {prontuario}")
        print(f"{'='*80}")
        print(f"- view_micromanipulacao: {len(df_micromanipulacao)} records")
        print(f"- view_micromanipulacao_oocitos: {len(df_oocitos)} records")
        print(f"- view_tratamentos: {len(df_tratamentos)} records")
        print(f"- view_congelamentos_embrioes: {len(df_congelamentos_embrioes)} records")
        print(f"- view_descongelamentos_embrioes: {len(df_descongelamentos_embrioes)} records")
        print(f"- view_embrioes_congelados: {len(df_embrioes_congelados)} records")
        print(f"- view_congelamentos_ovulos: {len(df_congelamentos_ovulos)} records")
        print(f"- view_descongelamentos_ovulos: {len(df_descongelamentos_ovulos)} records")
        print(f"- view_ovulos_congelados: {len(df_ovulos_congelados)} records")
        print(f"- gold.clinisys_embrioes: {len(df_clinisys_embrioes)} records")
        print(f"- gold.embryoscope_embrioes: {len(df_embryoscope_embrioes)} records")
        print(f"- gold.embryoscope_clinisys_combined: {len(df_embryoscope_clinisys_combined)} records")
        print(f"- silver.planilha_embriologia: {len(df_planilha_embriologia)} records")
        print(f"- gold.planilha_embryoscope_combined: {len(df_planilha_embryoscope_combined)} records")
        print(f"{'='*80}")
        
        # Return results as dictionary
        results = {
            'micromanipulacao': df_micromanipulacao,
            'micromanipulacao_oocitos': df_oocitos,
            'tratamentos': df_tratamentos,
            'congelamentos_embrioes': df_congelamentos_embrioes,
            'descongelamentos_embrioes': df_descongelamentos_embrioes,
            'embrioes_congelados': df_embrioes_congelados,
            'congelamentos_ovulos': df_congelamentos_ovulos,
            'descongelamentos_ovulos': df_descongelamentos_ovulos,
            'ovulos_congelados': df_ovulos_congelados,
            'clinisys_embrioes': df_clinisys_embrioes,
            'embryoscope_embrioes': df_embryoscope_embrioes,
            'embryoscope_clinisys_combined': df_embryoscope_clinisys_combined,
            'planilha_embriologia': df_planilha_embriologia,
            'planilha_embryoscope_combined': df_planilha_embryoscope_combined
        }
        
        return results
        
    finally:
        # Close connections
        clinisys_con.close()
        huntington_con.close()
        print("\nDatabase connections closed.")


## Example Usage


In [65]:
# Example usage
prontuario = 818375 # não exitem dados em lugar nenhum
prontuario = 138475
# Query all embryo data
results = query_embryo_data(prontuario)



QUERYING EMBRYO DATA FOR PRONTUARIO: 138475
Query executed at: 2025-12-19 13:29:54.302035

=== VIEW_MICROMANIPULACAO ===
Found 3 records in view_micromanipulacao

=== VIEW_MICROMANIPULACAO_OOCITOS ===
Found 37 records in view_micromanipulacao_oocitos

=== VIEW_TRATAMENTOS ===
Found 5 records in view_tratamentos

=== VIEW_CONGELAMENTOS_EMBRIOES ===
Found 3 records in view_congelamentos_embrioes

=== VIEW_DESCONGELAMENTOS_EMBRIOES ===
Found 3 records in view_descongelamentos_embrioes

=== VIEW_EMBRIOES_CONGELADOS ===
Found 11 records in view_embrioes_congelados

=== VIEW_CONGELAMENTOS_OVULOS ===
Found 1 records in view_congelamentos_ovulos

=== VIEW_DESCONGELAMENTOS_OVULOS ===
Found 2 records in view_descongelamentos_ovulos

=== VIEW_OVULOS_CONGELADOS ===
Found 22 records in view_ovulos_congelados

=== GOLD.CLINISYS_EMBRIOES ===
Found 38 records in gold.clinisys_embrioes

=== GOLD.EMBRYOSCOPE_EMBRIOES ===
Found 32 records in gold.embryoscope_embrioes

=== GOLD.EMBRYOSCOPE_CLINISYS_COMBI

## Display Results

### 1. Micromanipulacao


In [66]:
print("=== MICROMANIPULACAO DATA ===")
micromanipulacao_data = results['micromanipulacao']
print(f"Records: {len(micromanipulacao_data)}")
if not micromanipulacao_data.empty:
    display(micromanipulacao_data)
else:
    print("No micromanipulacao data found.")


=== MICROMANIPULACAO DATA ===
Records: 3


,codigo_ficha,numero_caso,prontuario,IdadeEsposa_DG,IdadeMarido_DG,Data_DL,horario_inicial_fert,horario_final_fert,Aspiracao_DL,medico_id,responsavel_puncao_id,recepcao_ovulos,Denud_DL,checagem_fertilizacao,checagem_d3,checagem_d5,checagem_d6,checagem_d7,foliculos,oocitos,ICSIDescongelados,pgd_oocito,pgd_d5,data_procedimento,tecnica_injetados,motilidade,morfologia_injetados,gradiente_descontinuo,unidade_id,responsavel_denud_id,profissional_fertilizacao_id,profissional_d3_id,profissional_d5_id,profissional_d6_id,profissional_d7_id,materiais_insumos,local_coleta,viscosidade,volume_amostra_utilizada,morfologia_pre,morfologia_pos,zymot,observacoes_resumo_ciclos,agulha_icsi,lote_agulha_icsi,validade_agulha_icsi,agulha_aspir,lote_agulha_aspir,validade_agulha_aspir,incubadora,abstinencia_dias,meios,responsavel_oocitos_id,fonte,inicio_ps,termino_ps,volume_ps1,CongelamentoOvulos,concentracao_progressivos_pre,conc_ps1,EstadoSptz,check_gradiente_ps,check_swimup,volume_ps2,concentracao_progressivos_pos,conc_ps2,comentarios_ps,responsavel_ps,timelapse,transferidos,Transfer_D5,medico_responsavel_id,azoospermia,Observacoes,responsavel_labfiv,responsavel_labfiv_data,tipo_paciente,alteracoes_oocitarias_normal,alteracoes_oocitarias_granulacao,alteracoes_oocitarias_pelucida,alteracoes_oocitarias_oval,alteracoes_oocitarias_inclusoes,alteracoes_oocitarias_pervitelineo_aumentado,alteracoes_oocitarias_irregular,alteracoes_oocitarias_necrotica,alteracoes_oocitarias_pervitelineo_granular,alteracoes_oocitarias_ser,alteracoes_oocitarias_corpusculo_frag,alteracoes_oocitarias_vacuolos,alteracoes_oocitarias_corpusculo_aum,alteracoes_oocitarias_membrana,alteracoes_oocitarias_corpusculo_peq,alteracoes_oocitarias_citoplasma,alteracoes_oocitarias_corpusculo_deg,controle_anual,maia,hash,extraction_timestamp
0,21961,2764/24,138475,41,NaN,2024-08-20,12:30,12:45,09:25,31,3692,Não,10:50,08:00,08:00,08:00,08:00,None,18,15,Não,1,1,2024-08-20,ICSI,14 espermatozoides móveis progressivos\r\n,14 espermatozoides normais\r\n,1,0000000001,3652,4318,4318,4005,3505,None,None,Não se aplica,None,NA,NA,04,None,None,None,None,None,None,None,None,ES 2,NA,None,None,Ejaculado Doador,11:13,12:05,NA,Não,NA,NA,Criopreservado,None,None,"0,2","1,0","8,0",Descongelada 01 (uma) vial de amostra seminal de doador sob a identificação FF7018. Amostra recongelada sob mesma identificação.\r\nResta 01 (uma) vial de amostra seminal de doador sob a identificação FF7018.,4003,Sim,0,Sim,31,Nenhum,None,None,None,esposa,14,14,0,0,14,0,0,0,0,0,1,0,0,0,0,0,0,IB1350/24,Sim,ae0ba4929befc6835b038721c88987c5,2025-07-21 21:39:47
1,19397,451/24,138475,40,NaN,2024-02-16,13:20,13:50,None,31,None,Não,None,08:00,08:00,08:00,None,None,None,None,Sim,None,None,2024-02-16,ICSI,09 espermatozoides móveis progressivos\r\n,09 espermatozoides normais\r\n,None,0000000001,None,3505,5997,None,None,None,None,Não se aplica,None,NA,NA,04,None,None,None,None,None,None,None,None,ES 1,NA,None,None,Ejaculado Doador,08:26,09:07,NA,None,NA,NA,Criopreservado,None,None,"0,2","2,0","10,0",Descongelada 01 (uma) vial de amostra seminal de doador sob identificação FF6451. Não há mais amostras criopreservadas.,3529,Sim,0,Sim,31,Nenhum,None,None,None,esposa,9,9,0,0,9,1,0,0,0,0,2,0,0,0,0,0,0,IB221/24,None,9610bd25e8c6b143577128ead2f6c8f9,2025-07-21 21:39:47
2,16457,1446/23,138475,40,NaN,2023-07-16,12:40,13:10,None,31,None,Não,None,07:00,08:00,08:00,None,None,None,11,Sim,None,None,2023-07-16,ICSI,09 espermatozoides móveis progressivos\r\n,09 espermatozoides normais\r\n,1,0000000001,None,3613,5997,5997,None,None,None,Não se aplica,None,NA,NA,04,None,None,None,None,None,None,None,None,None,NA,None,None,Ejaculado Doador,07:53,08:52,NA,None,NA,NA,Criopreservado,None,None,"0,5","8,0","17,0",Descongelada 01 (uma) vial de amostra de doador sob a identificação FF6451. Amostra recongelada sob a mesma identificação.,4003,Sim,1,Sim,31,Nenhum,None,None,None,esposa,8,9,0,0,9,0,1,0,0,0,2,0,0,1,0,0,1,IB1173/23,None,eb4426eec93b16af6df28a9efba14804,2025-07-21 

### 2. Micromanipulacao Oocitos


In [67]:
print("=== MICROMANIPULACAO OOCITOS DATA ===")
oocitos_data = results['micromanipulacao_oocitos']
print(f"Records: {len(oocitos_data)}")
if not oocitos_data.empty:
    display(oocitos_data)
else:
    print("No micromanipulacao oocitos data found.")


=== MICROMANIPULACAO OOCITOS DATA ===
Records: 37


,id,id_micromanipulacao,diaseguinte,Maturidade,Embriologista,PI,TCD,PGD,ResultadoPGD,IdentificacaoPGD,Fertilizacao,CorpusculoPolar,ComentariosDepois,OocitoDoado,OrigemOocito,InseminacaoOocito,NClivou_D3,NCelulas_D3,Frag_D3,Blastomero_D3,NClivou_D4,NCelulas_D4,Compactando_D4,MassaInterna_D4,Trofoblasto_D4,NClivou_D5,NCelulas_D5,Compactando_D5,MassaInterna_D5,Trofoblasto_D5,NClivou_D6,NCelulas_D6,Compactando_D6,MassaInterna_D6,Trofoblasto_D6,NClivou_D7,NCelulas_D7,Compactando_D7,MassaInterna_D7,Trofoblasto_D7,score_maia,relatorio_ia,hash,extraction_timestamp,flag_embryoscope,embryo_number
0,153421,16457,Não,MII,3548,None,Criopreservado,Não,None,None,2PN,None,Criopreservado,Não,Descongelado,ICSI,Sim,8,3,Simétricos,None,None,None,None,None,Sim,None,Blastocisto Grau 4,B,B,None,None,None,None,None,None,None,None,None,None,None,None,82fad5a35bf1d962ca8d1f09e6335cde,2025-07-21 21:40:02,1,1
1,153422,16457,Não,MII,3548,None,Descartado,None,None,None,2PN,None,Criopreservado,Não,Descongelado,ICSI,Sim,3,3,Simétricos,None,None,None,None,None,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,cb3f59600d678a0a19b21066859fbcf5,2025-07-21 21:40:02,1,2
2,153423,16457,Não,MII,3548,None,Descartado,None,None,None,2PN,None,Criopreservado,Não,Descongelado,ICSI,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,5098c0ad88461dcf5c1b5ae9fa5fa7bf,2025-07-21 21:40:02,1,3
3,153424,16457,Não,MII,3548,None,Transferido,Não,None,None,2PN,None,Criopreservado,Não,Descongelado,ICSI,Sim,8,2,Simétricos,None,None,None,None,None,Sim,None,Blastocisto Grau 4,A,A,None,None,None,None,None,None,None,None,None,None,None,None,5ff023bdadfdc25cf80b2aef072c5ad1,2025-07-21 21:40:02,1,4
4,153425,16457,Não,MII,3548,None,Descartado,None,None,None,1PN 2CP,None,Criopreservado,Não,Descongelado,ICSI,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,9091c16f45238d3f4b5c20ab58247f89,2025-07-21 21:40:02,1,5
5,153426,16457,Não,MII,3548,None,Descartado,None,None,None,2CP,None,Criopreservado,Não,Descongelado,ICSI,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0d15706d704b75e98128ea90d31e4c61,2025-07-21 21:40:02,1,6
6,153427,16457,Não,MII,3548,None,Descartado,None,None,None,1CP,None,Criopreservado,Não,Descongelado,ICSI,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,a764698dffce4bf43edd8ca1548f0833,2025-07-21 21:40:02,1,7
7,153428,16457,Não,MII,3548,None,Descartado,None,None,None,2PN,None,Criopreservado,Não,Descongelado,ICSI,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,eca00de5f6a8191f540097d705b7c05d,2025-07-21 21:40:02,1,8
8,153429,16457,Não,MII,3548,None,Descartado,None,None,None,1CP,None,Criopreservado,Não,Descongelado,ICSI,Sim,2,2,Simétricos,None,None,None,None,None,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,fd6308f0d78f98ac6eb9951442016d82,2025-07-21 21:40:02,1,9
9,153430,16457,Não,DEG,None,None,Descartado,None,None,None,None,None,None,Não,Descongelado,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,44c2ca4ace5b5fcca38934180ffecd7c,2025-07-21 21:40:02,0,10


### 3. Tratamentos


In [68]:
print("=== TRATAMENTOS DATA ===")
tratamentos_data = results['tratamentos']
print(f"Records: {len(tratamentos_data)}")
if not tratamentos_data.empty:
    display(tratamentos_data)
else:
    print("No tratamentos data found.")


=== TRATAMENTOS DATA ===
Records: 5


,id,prontuario,unidade,idade_esposa,idade_marido,paciente_tratamento,tentativa,data_procedimento,tipo_procedimento,tipo_inducao,protocolo,tipo_preparo_endometrial,origem_ovulo,status_ovulo,prontuario_doadora,origem_espermatozoide,fonte_espermatozoide,status_espermatozoide,origem_material,utero_substituicao,prontuario_genitores,doacao_ovulos,doacao_embrioes,pgt,era_auxiliar,emma_resultado,alice_resultado,picsi,hatching_od,hatching_oe,medicacao_parental,peso_paciente,altura_paciente,fumante_paciente,data_dum,peso_conjuge,altura_conjuge,fumante_conjuge,fator_infertilidade1,fator_infertilidade2,foliculos_antrais,data_inicio_inducao,dias_inducao_preparo,paciente_inducao,data_inicio_agonista,utilizado_hcg,data_hcg,horario_planejado_hcg,gonapeptyl,dose_gonapeptyl,data_inicio_progesterona,progesterona_vaginal,data_congelamento,metodo_descongelamento,valor_palheta,manter_excedentes,dia_transferencia,data_transferencia,medida_endometrio,padrao_endometrio,motivo_nao_transferir,resultado_tratamento,analise_genetica,altura_bebe1,via_parto_bebe1,altura_bebe2,via_parto_bebe2,altura_bebe3,via_parto_bebe3,altura_bebe4,via_parto_bebe4,observacoes_gerais,usuario_responsavel,responsavel_informacoes,hash,extraction_timestamp
0,30064,138475,1,41,NaN,esposa,5,2024-11-10,Ciclo de Congelados,None,None,Hormonal,None,None,NaN,None,None,None,Heterólogo,Não,NaN,None,None,Não,Não,Não,Não,None,None,None,None,60.0,1.6,Não,2024-10-19,NaN,NaN,None,Outros,None,None,2024-10-22,20,esposa,None,Não utilizado,None,None,None,None,None,None,NaT,None,None,None,5,2024-11-10,None,None,None,Gestação Clínica,Não,None,None,None,None,None,None,None,None,None,3673,31,f6d7b5b2eabfa3c624a524235cbb1ef6,2025-07-21 21:38:38
1,28530,138475,1,41,NaN,esposa,4,2024-08-20,Ciclo a Fresco FIV,Segunda onda,None,Hormonal,Homólogo,Fresco,NaN,Heterólogo,Sêmen,Criopreservado,None,Não,NaN,Não,Não,PGT-A,None,None,None,None,None,None,None,60.0,1.6,Não,2024-07-11,NaN,NaN,None,Outros,None,None,2024-08-07,None,esposa,None,Não utilizado,None,None,None,None,None,None,NaT,None,None,None,No transfer,NaT,None,None,Criopreservação total,No transfer,Não,None,None,None,None,None,None,None,None,None,3719,31,880cad13373d8f10b5eddffa40196703,2025-07-21 21:38:38
2,27422,138475,1,40,NaN,esposa,3,2024-06-27,Ciclo de Congelados,None,None,Hormonal,None,None,NaN,None,None,None,Homólogo,Não,NaN,None,None,PGT-A,Não,None,None,None,None,None,None,60.0,1.6,Não,2024-06-06,NaN,NaN,None,Outros,None,None,2024-06-07,21,esposa,None,Não utilizado,None,None,None,None,None,None,NaT,None,None,None,5,2024-06-27,None,None,None,Negativo,Não,None,None,None,None,None,None,None,None,None,3765,31,a82909fca065e1c77613f3b83c78b293,2025-07-21 21:38:38
3,22820,138475,1,40,NaN,esposa,2,2023-10-18,Ciclo de Congelados,None,None,Natural,None,None,NaN,None,None,None,Heterólogo,Não,NaN,None,None,Não,Não,None,None,None,None,None,None,60.0,1.6,Não,2023-10-02,NaN,NaN,None,Outros,None,None,2023-10-04,15,esposa,None,Não utilizado,None,None,None,None,None,None,NaT,None,None,None,5,2023-10-18,None,None,None,Gestação Clínica,Sim,None,None,None,None,None,None,None,None,None,3765,31,ecabf14085988bf1a21b624ef6b52681,2025-07-21 21:38:38
4,20892,138475,1,40,NaN,esposa,1,NaT,Ciclo a Fresco FIV,Convencional,None,Hormonal,Homólogo,Criopreservado,NaN,Heterólogo,Sêmen,Criopreservado,None,Não,NaN,Não,Não,Não,Não,Não,Não,None,None,Sim,None,60.0,1.6,Não,2023-06-29,NaN,NaN,None,Outros,None,None,2023-07-01,None,esposa,None,Não utilizado,None,None,None,None,None,None,NaT,None,None,None,None,NaT,None,None,None,Negativo,Não,0,0,0,0,0,0,0,0,None,3600,31,30560792f769b02b88fa30002f61032b,2025-07-21 21:38:38


### 4. Congelamentos Embrioes


In [69]:
print("=== CONGELAMENTOS EMBRIOES DATA ===")
congelamentos_data = results['congelamentos_embrioes']
print(f"Records: {len(congelamentos_data)}")
if not congelamentos_data.empty:
    display(congelamentos_data)
else:
    print("No congelamentos embrioes data found.")


=== CONGELAMENTOS EMBRIOES DATA ===
Records: 3


,id,CodCongelamento,Unidade,prontuario,paciente,Data,Hora,Ciclo,CicloRecongelamento,NEmbrioes,NPailletes,Identificacao,Tambor,Cane,Cane2,Tecnica,D5,D6,Observacoes,BiologoFIV,BiologoFIV2,status_financeiro,hash,extraction_timestamp
0,26420,E2183/24,1,138475,esposa,2024-08-25,1900-01-01 13:20:00,2764/24,Não,9,9,E21443 + E21446,None,None,None,Vitrificação,1.0,1.0,None,Bruna Lázaro Lourenço,Priscila Fabiane Dos Santos Beirigo Paternostro,None,1ebc2bf7296ce0a41edfa7176e0432e3,2025-07-21 21:38:47
1,24637,E413/24,1,138475,esposa,2024-02-21,1900-01-01 12:05:00,451/24,Não,1,1,E20739,AH,1,None,Vitrificação,1.0,NaN,None,Izadora Reis,Bruna Arroio,None,886ff26cba1cc52b3bda1c6c35e962a2,2025-07-21 21:38:47
2,22586,E20256/23,1,138475,esposa,2023-07-21,1900-01-01 14:21:00,1446/23,Não,1,1,E19906,SIGMA,2,None,Vitrificação,1.0,NaN,None,Izadora Reis,Priscila Fabiane Dos Santos Beirigo Paternostro,None,2b0561a4c507c38a58d323181ee4e919,2025-07-21 21:38:47


### 5. Descongelamentos Embrioes


In [70]:
print("=== DESCONGELAMENTOS EMBRIOES DATA ===")
descongelamentos_data = results['descongelamentos_embrioes']
print(f"Records: {len(descongelamentos_data)}")
if not descongelamentos_data.empty:
    display(descongelamentos_data)
else:
    print("No descongelamentos embrioes data found.")


=== DESCONGELAMENTOS EMBRIOES DATA ===
Records: 3


,id,CodDescongelamento,Unidade,prontuario,doadora,DataCongelamento,DataDescongelamento,Ciclo,Identificacao,CodCongelamento,Tambor,Cane,PailletesDescongeladas,Tecnica,DataTransferencia,Incubadora,transferidos_transferencia,cateter_transferencia,lote_transferencia,validade_transferencia,sangue_interno_transferencia,sangue_externo_transferencia,retorno_transferencia,Transfer_D5,responsavel_transferencia,Observacoes,BiologoFIV,BiologoFIV2,hash,extraction_timestamp
0,16300,2441/24,1,138475,0,2024-08-25,2024-11-10,2764/24,E21443 + E21446,E2183/24,OMICRON,2,1,DESVITRIFICAÇÃO,2024-11-10,12.3,1,SYDNEY,None,None,Não,Não,Não,None,3719.0,None,Beatriz Aiello,Dayane Reis,00f5f9b50febcc7c12f2e0e95b96ea9f,2025-07-21 21:38:51
1,15149,1343/24,1,138475,0,2024-02-21,2024-06-27,451/24,E20739,E413/24,AH,1,1,DESVITRIFICAÇÃO,2024-06-27,13.2,1,SYDNEY,None,None,Não,Não,Não,None,3719.0,None,Beatriz Aiello,Laura Belomo,caeb2e7301b85f8ca0ae7dba316ba64e,2025-07-21 21:38:51
2,13021,1645/23,1,138475,0,2023-07-21,2023-10-18,1446/23,E19906,E20256/23,SIGMA,2,1,DESVITRIFICAÇÃO,2023-10-18,13.9,1,WALLACE,None,None,Não,Não,Não,None,3719.0,None,Izadora Reis,Laura Belomo,23d66b776c67bd17bb5ae984ef275273,2025-07-21 21:38:51


### 6. Embrioes Congelados


In [71]:
print("=== EMBRIOES CONGELADOS DATA ===")
embrioes_congelados_data = results['embrioes_congelados']
print(f"Records: {len(embrioes_congelados_data)}")
if not embrioes_congelados_data.empty:
    # display(embrioes_congelados_data)
    display(embrioes_congelados_data[embrioes_congelados_data['id_descongelamento']!=0])
else:
    print("No embrioes congelados data found.")


=== EMBRIOES CONGELADOS DATA ===
Records: 11


,id,id_oocito,id_congelamento,id_descongelamento,prontuario,pailletes,pailletes_id,embriao,qualidade,comentarios,comentarios_descongelamento,ah,dia5_2,dia5_3,dia5_4,transferidos,pgd,colapso,pgd_congelamento,resultado_pgd_congelamento,mito_teste_pgd_congelamento,dia_congelamento,score_maia,tanque_amostra,caneca_amostra,rack_amostra,hash,extraction_timestamp
8,76266,212875,26420,16300,138475,P1,E21443,#1,Blastocisto Grau 5 - A - A,None,"EUPLOIDE, Feminino",Sim,Blastocisto Grau 5,A,A,Transferido,Sim,Normal/Euplóide XX,Sim,Euploide,9.2,D5,7.93,OMICRON,2,E21443,284667cb86fdb236e9898bbabdeed7e1,2025-07-21 21:38:57
9,69843,184181,24637,15149,138475,P9,E20739,#9,Blastocisto Grau 4 - B - A,None,"EUPLOIDE, Masculino",Sim,Blastocisto Grau 4,B,A,Transferido,Sim,Normal/Euplóide Masculino,Sim,Normal,8.3,D5,7.26,None,None,None,cda9d98470afbef8a5b4be49c84eb3cf,2025-07-21 21:38:57
10,62728,153421,22586,13021,138475,P1,None,(1),Blastocisto Grau 4 - B - B,None,NÃO BIOPSIADO / SCORE 6.0,Não,Blastocisto Grau 4,B,B,Transferido,Não,None,Não,None,6.0,D5,None,None,None,None,d9adc076e5013865b997f43a03297375,2025-07-21 21:38:57


### 7. Clinisys Embrioes


In [72]:
print("=== CLINISYS EMBRIOES DATA ===")
clinisys_embrioes_data = results['clinisys_embrioes']
print(f"Records: {len(clinisys_embrioes_data)}")
if not clinisys_embrioes_data.empty:
    display(clinisys_embrioes_data.head(10))
else:
    print("No clinisys embrioes data found.")


=== CLINISYS EMBRIOES DATA ===
Records: 38


,oocito_id,oocito_id_micromanipulacao,oocito_diaseguinte,oocito_Maturidade,oocito_Embriologista,oocito_PI,oocito_TCD,oocito_PGD,oocito_ResultadoPGD,oocito_IdentificacaoPGD,oocito_Fertilizacao,oocito_CorpusculoPolar,oocito_ComentariosDepois,oocito_OocitoDoado,oocito_OrigemOocito,oocito_InseminacaoOocito,oocito_NClivou_D3,oocito_NCelulas_D3,oocito_Frag_D3,oocito_Blastomero_D3,oocito_NClivou_D4,oocito_NCelulas_D4,oocito_Compactando_D4,oocito_MassaInterna_D4,oocito_Trofoblasto_D4,oocito_NClivou_D5,oocito_NCelulas_D5,oocito_Compactando_D5,oocito_MassaInterna_D5,oocito_Trofoblasto_D5,oocito_NClivou_D6,oocito_NCelulas_D6,oocito_Compactando_D6,oocito_MassaInterna_D6,oocito_Trofoblasto_D6,oocito_NClivou_D7,oocito_NCelulas_D7,oocito_Compactando_D7,oocito_MassaInterna_D7,oocito_Trofoblasto_D7,oocito_score_maia,oocito_relatorio_ia,oocito_flag_embryoscope,oocito_embryo_number,micro_codigo_ficha,micro_numero_caso,micro_prontuario,micro_IdadeEsposa_DG,micro_IdadeMarido_DG,micro_Data_DL,micro_horario_inicial_fert,micro_horario_final_fert,micro_Aspiracao_DL,micro_medico_id,micro_responsavel_puncao_id,micro_recepcao_ovulos,micro_Denud_DL,micro_checagem_fertilizacao,micro_checagem_d3,micro_checagem_d5,micro_checagem_d6,micro_checagem_d7,micro_foliculos,micro_oocitos,micro_ICSIDescongelados,micro_pgd_oocito,micro_pgd_d5,micro_data_procedimento,micro_tecnica_injetados,micro_motilidade,micro_morfologia_injetados,micro_gradiente_descontinuo,micro_unidade_id,micro_responsavel_denud_id,micro_profissional_fertilizacao_id,micro_profissional_d3_id,micro_profissional_d5_id,micro_profissional_d6_id,micro_profissional_d7_id,micro_materiais_insumos,micro_local_coleta,micro_viscosidade,micro_volume_amostra_utilizada,micro_morfologia_pre,micro_morfologia_pos,micro_zymot,micro_observacoes_resumo_ciclos,micro_agulha_icsi,micro_lote_agulha_icsi,micro_validade_agulha_icsi,micro_agulha_aspir,micro_lote_agulha_aspir,micro_validade_agulha_aspir,micro_incubadora,micro_abstinencia_dias,micro_meios,micro_responsavel_oocitos_id,micro_fonte,micro_inicio_ps,micro_termino_ps,micro_volume_ps1,micro_CongelamentoOvulos,micro_concentracao_progressivos_pre,micro_conc_ps1,micro_EstadoSptz,micro_check_gradiente_ps,micro_check_swimup,micro_volume_ps2,micro_concentracao_progressivos_pos,micro_conc_ps2,micro_comentarios_ps,micro_responsavel_ps,micro_timelapse,micro_transferidos,micro_Transfer_D5,micro_medico_responsavel_id,micro_azoospermia,micro_Observacoes,micro_responsavel_labfiv,micro_responsavel_labfiv_data,micro_tipo_paciente,micro_alteracoes_oocitarias_normal,micro_alteracoes_oocitarias_granulacao,micro_alteracoes_oocitarias_pelucida,micro_alteracoes_oocitarias_oval,micro_alteracoes_oocitarias_inclusoes,micro_alteracoes_oocitarias_pervitelineo_aumentado,micro_alteracoes_oocitarias_irregular,micro_alteracoes_oocitarias_necrotica,micro_alteracoes_oocitarias_pervitelineo_granular,micro_alteracoes_oocitarias_ser,micro_alteracoes_oocitarias_corpusculo_frag,micro_alteracoes_oocitarias_vacuolos,micro_alteracoes_oocitarias_corpusculo_aum,micro_alteracoes_oocitarias_membrana,micro_alteracoes_oocitarias_corpusculo_peq,micro_alteracoes_oocitarias_citoplasma,micro_alteracoes_oocitarias_corpusculo_deg,micro_controle_anual,micro_maia,cong_em_id,cong_em_CodCongelamento,cong_em_Unidade,cong_em_prontuario,cong_em_paciente,cong_em_Data,cong_em_Hora,cong_em_Ciclo,cong_em_CicloRecongelamento,cong_em_NEmbrioes,cong_em_NPailletes,cong_em_Identificacao,cong_em_Tambor,cong_em_Cane,cong_em_Cane2,cong_em_Tecnica,cong_em_D5,cong_em_D6,cong_em_Observacoes,cong_em_BiologoFIV,cong_em_BiologoFIV2,cong_em_status_financeiro,descong_em_id,descong_em_CodDescongelamento,descong_em_Unidade,descong_em_prontuario,descong_em_doadora,descong_em_DataCongelamento,descong_em_DataDescongelamento,descong_em_Ciclo,descong_em_Identificacao,descong_em_CodCongelamento,descong_em_Tambor,descong_em_Cane,descong_em_PailletesDescongeladas,descong_em_Tecnica,descong_em_DataTransferencia,descong_em_Incubadora,descong_em_transferido

In [73]:
clinisys_embrioes_data[clinisys_embrioes_data['oocito_id'].isin([184181])]

,oocito_id,oocito_id_micromanipulacao,oocito_diaseguinte,oocito_Maturidade,oocito_Embriologista,oocito_PI,oocito_TCD,oocito_PGD,oocito_ResultadoPGD,oocito_IdentificacaoPGD,oocito_Fertilizacao,oocito_CorpusculoPolar,oocito_ComentariosDepois,oocito_OocitoDoado,oocito_OrigemOocito,oocito_InseminacaoOocito,oocito_NClivou_D3,oocito_NCelulas_D3,oocito_Frag_D3,oocito_Blastomero_D3,oocito_NClivou_D4,oocito_NCelulas_D4,oocito_Compactando_D4,oocito_MassaInterna_D4,oocito_Trofoblasto_D4,oocito_NClivou_D5,oocito_NCelulas_D5,oocito_Compactando_D5,oocito_MassaInterna_D5,oocito_Trofoblasto_D5,oocito_NClivou_D6,oocito_NCelulas_D6,oocito_Compactando_D6,oocito_MassaInterna_D6,oocito_Trofoblasto_D6,oocito_NClivou_D7,oocito_NCelulas_D7,oocito_Compactando_D7,oocito_MassaInterna_D7,oocito_Trofoblasto_D7,oocito_score_maia,oocito_relatorio_ia,oocito_flag_embryoscope,oocito_embryo_number,micro_codigo_ficha,micro_numero_caso,micro_prontuario,micro_IdadeEsposa_DG,micro_IdadeMarido_DG,micro_Data_DL,micro_horario_inicial_fert,micro_horario_final_fert,micro_Aspiracao_DL,micro_medico_id,micro_responsavel_puncao_id,micro_recepcao_ovulos,micro_Denud_DL,micro_checagem_fertilizacao,micro_checagem_d3,micro_checagem_d5,micro_checagem_d6,micro_checagem_d7,micro_foliculos,micro_oocitos,micro_ICSIDescongelados,micro_pgd_oocito,micro_pgd_d5,micro_data_procedimento,micro_tecnica_injetados,micro_motilidade,micro_morfologia_injetados,micro_gradiente_descontinuo,micro_unidade_id,micro_responsavel_denud_id,micro_profissional_fertilizacao_id,micro_profissional_d3_id,micro_profissional_d5_id,micro_profissional_d6_id,micro_profissional_d7_id,micro_materiais_insumos,micro_local_coleta,micro_viscosidade,micro_volume_amostra_utilizada,micro_morfologia_pre,micro_morfologia_pos,micro_zymot,micro_observacoes_resumo_ciclos,micro_agulha_icsi,micro_lote_agulha_icsi,micro_validade_agulha_icsi,micro_agulha_aspir,micro_lote_agulha_aspir,micro_validade_agulha_aspir,micro_incubadora,micro_abstinencia_dias,micro_meios,micro_responsavel_oocitos_id,micro_fonte,micro_inicio_ps,micro_termino_ps,micro_volume_ps1,micro_CongelamentoOvulos,micro_concentracao_progressivos_pre,micro_conc_ps1,micro_EstadoSptz,micro_check_gradiente_ps,micro_check_swimup,micro_volume_ps2,micro_concentracao_progressivos_pos,micro_conc_ps2,micro_comentarios_ps,micro_responsavel_ps,micro_timelapse,micro_transferidos,micro_Transfer_D5,micro_medico_responsavel_id,micro_azoospermia,micro_Observacoes,micro_responsavel_labfiv,micro_responsavel_labfiv_data,micro_tipo_paciente,micro_alteracoes_oocitarias_normal,micro_alteracoes_oocitarias_granulacao,micro_alteracoes_oocitarias_pelucida,micro_alteracoes_oocitarias_oval,micro_alteracoes_oocitarias_inclusoes,micro_alteracoes_oocitarias_pervitelineo_aumentado,micro_alteracoes_oocitarias_irregular,micro_alteracoes_oocitarias_necrotica,micro_alteracoes_oocitarias_pervitelineo_granular,micro_alteracoes_oocitarias_ser,micro_alteracoes_oocitarias_corpusculo_frag,micro_alteracoes_oocitarias_vacuolos,micro_alteracoes_oocitarias_corpusculo_aum,micro_alteracoes_oocitarias_membrana,micro_alteracoes_oocitarias_corpusculo_peq,micro_alteracoes_oocitarias_citoplasma,micro_alteracoes_oocitarias_corpusculo_deg,micro_controle_anual,micro_maia,cong_em_id,cong_em_CodCongelamento,cong_em_Unidade,cong_em_prontuario,cong_em_paciente,cong_em_Data,cong_em_Hora,cong_em_Ciclo,cong_em_CicloRecongelamento,cong_em_NEmbrioes,cong_em_NPailletes,cong_em_Identificacao,cong_em_Tambor,cong_em_Cane,cong_em_Cane2,cong_em_Tecnica,cong_em_D5,cong_em_D6,cong_em_Observacoes,cong_em_BiologoFIV,cong_em_BiologoFIV2,cong_em_status_financeiro,descong_em_id,descong_em_CodDescongelamento,descong_em_Unidade,descong_em_prontuario,descong_em_doadora,descong_em_DataCongelamento,descong_em_DataDescongelamento,descong_em_Ciclo,descong_em_Identificacao,descong_em_CodCongelamento,descong_em_Tambor,descong_em_Cane,descong_em_PailletesDescongeladas,descong_em_Tecnica,descong_em_DataTransferencia,descong_em_Incubadora,descong_em_transferido

In [74]:
colunas = [
    'oocito_id','oocito_id_micromanipulacao', 'oocito_embryo_number', 'oocito_TCD', 
    'micro_codigo_ficha', 'micro_numero_caso', 'micro_Data_DL', 'micro_data_procedimento', 
    'cong_em_id', 'cong_em_CodCongelamento', 'cong_em_Ciclo', 'cong_em_NEmbrioes', 
    'emb_cong_id', 'emb_cong_id_oocito', 'emb_cong_id_congelamento', 'emb_cong_id_descongelamento','emb_cong_embriao', 
    'descong_em_id', 'descong_em_transferidos_transferencia'	, 'descong_em_CodDescongelamento', 'descong_em_Ciclo' , 'descong_em_DataDescongelamento', 'descong_em_DataTransferencia', 
    'trat_id', 'trat_data_procedimento', 'trat_data_transferencia', 'trat_resultado_tratamento'
]

clinisys_embrioes_data.loc[clinisys_embrioes_data['oocito_embryo_number'].isin([33,35]), colunas]

,oocito_id,oocito_id_micromanipulacao,oocito_embryo_number,oocito_TCD,micro_codigo_ficha,micro_numero_caso,micro_Data_DL,micro_data_procedimento,cong_em_id,cong_em_CodCongelamento,cong_em_Ciclo,cong_em_NEmbrioes,emb_cong_id,emb_cong_id_oocito,emb_cong_id_congelamento,emb_cong_id_descongelamento,emb_cong_embriao,descong_em_id,descong_em_transferidos_transferencia,descong_em_CodDescongelamento,descong_em_Ciclo,descong_em_DataDescongelamento,descong_em_DataTransferencia,trat_id,trat_data_procedimento,trat_data_transferencia,trat_resultado_tratamento


### 8. Embryoscope Embrioes


In [75]:
print("=== EMBRYOSCOPE EMBRIOES DATA ===")
embryoscope_embrioes_data = results['embryoscope_embrioes']
print(f"Records: {len(embryoscope_embrioes_data)}")
if not embryoscope_embrioes_data.empty:
    display(embryoscope_embrioes_data)
else:
    print("No embryoscope embrioes data found.")


=== EMBRYOSCOPE EMBRIOES DATA ===
Records: 32


,patient_PatientIDx,patient_PatientID,patient_FirstName,patient_LastName,patient_DateOfBirth,prontuario,patient_unit_huntington,treatment_PatientIDx,treatment_TreatmentName,treatment_unit_huntington,embryo_EmbryoID,embryo_KIDDate,embryo_KIDScore,embryo_KIDUser,embryo_KIDVersion,embryo_Description,embryo_EmbryoDescriptionID,embryo_EmbryoFate,embryo_FertilizationMethod,embryo_FertilizationTime,embryo_InstrumentNumber,embryo_Position,embryo_WellNumber,embryo_embryo_number,embryo_unit_huntington,embryo_Name_BlastExpandLast,embryo_Time_BlastExpandLast,embryo_Timestamp_BlastExpandLast,embryo_Value_BlastExpandLast,embryo_Name_BlastomereSize,embryo_Time_BlastomereSize,embryo_Timestamp_BlastomereSize,embryo_Value_BlastomereSize,embryo_Name_EVEN2,embryo_Time_EVEN2,embryo_Timestamp_EVEN2,embryo_Value_EVEN2,embryo_Name_EVEN4,embryo_Time_EVEN4,embryo_Timestamp_EVEN4,embryo_Value_EVEN4,embryo_Name_EVEN8,embryo_Time_EVEN8,embryo_Timestamp_EVEN8,embryo_Value_EVEN8,embryo_Name_FRAG2,embryo_Time_FRAG2,embryo_Timestamp_FRAG2,embryo_Value_FRAG2,embryo_Name_FRAG2CAT,embryo_Time_FRAG2CAT,embryo_Timestamp_FRAG2CAT,embryo_Value_FRAG2CAT,embryo_Name_FRAG4,embryo_Time_FRAG4,embryo_Timestamp_FRAG4,embryo_Value_FRAG4,embryo_Name_FRAG8,embryo_Time_FRAG8,embryo_Timestamp_FRAG8,embryo_Value_FRAG8,embryo_Name_Fragmentation,embryo_Time_Fragmentation,embryo_Timestamp_Fragmentation,embryo_Value_Fragmentation,embryo_Name_ICM,embryo_Time_ICM,embryo_Timestamp_ICM,embryo_Value_ICM,embryo_Name_Line,embryo_Time_Line,embryo_Timestamp_Line,embryo_Value_Line,embryo_Name_MN2Type,embryo_Time_MN2Type,embryo_Timestamp_MN2Type,embryo_Value_MN2Type,embryo_Name_MorphologicalGrade,embryo_Time_MorphologicalGrade,embryo_Timestamp_MorphologicalGrade,embryo_Value_MorphologicalGrade,embryo_Name_MultiNucleation,embryo_Time_MultiNucleation,embryo_Timestamp_MultiNucleation,embryo_Value_MultiNucleation,embryo_Name_Nuclei2,embryo_Time_Nuclei2,embryo_Timestamp_Nuclei2,embryo_Value_Nuclei2,embryo_Name_PN,embryo_Time_PN,embryo_Timestamp_PN,embryo_Value_PN,embryo_Name_Pulsing,embryo_Time_Pulsing,embryo_Timestamp_Pulsing,embryo_Value_Pulsing,embryo_Name_ReexpansionCount,embryo_Time_ReexpansionCount,embryo_Timestamp_ReexpansionCount,embryo_Value_ReexpansionCount,embryo_Name_Strings,embryo_Time_Strings,embryo_Timestamp_Strings,embryo_Value_Strings,embryo_Name_TE,embryo_Time_TE,embryo_Timestamp_TE,embryo_Value_TE,embryo_Name_t2,embryo_Time_t2,embryo_Timestamp_t2,embryo_Value_t2,embryo_Name_t3,embryo_Time_t3,embryo_Timestamp_t3,embryo_Value_t3,embryo_Name_t4,embryo_Time_t4,embryo_Timestamp_t4,embryo_Value_t4,embryo_Name_t5,embryo_Time_t5,embryo_Timestamp_t5,embryo_Value_t5,embryo_Name_t6,embryo_Time_t6,embryo_Timestamp_t6,embryo_Value_t6,embryo_Name_t7,embryo_Time_t7,embryo_Timestamp_t7,embryo_Value_t7,embryo_Name_t8,embryo_Time_t8,embryo_Timestamp_t8,embryo_Value_t8,embryo_Name_t9,embryo_Time_t9,embryo_Timestamp_t9,embryo_Value_t9,embryo_Name_tB,embryo_Time_tB,embryo_Timestamp_tB,embryo_Value_tB,embryo_Name_tEB,embryo_Time_tEB,embryo_Timestamp_tEB,embryo_Value_tEB,embryo_Name_tHB,embryo_Time_tHB,embryo_Timestamp_tHB,embryo_Value_tHB,embryo_Name_tM,embryo_Time_tM,embryo_Timestamp_tM,embryo_Value_tM,embryo_Name_tPB2,embryo_Time_tPB2,embryo_Timestamp_tPB2,embryo_Value_tPB2,embryo_Name_tPNa,embryo_Time_tPNa,embryo_Timestamp_tPNa,embryo_Value_tPNa,embryo_Name_tPNf,embryo_Time_tPNf,embryo_Timestamp_tPNf,embryo_Value_tPNf,embryo_Name_tSB,embryo_Time_tSB,embryo_Timestamp_tSB,embryo_Value_tSB,embryo_Name_tSC,embryo_Time_tSC,embryo_Timestamp_tSC,embryo_Value_tSC,embryo_PatientIDx,embryo_TreatmentName
0,PC1P7BHG_45123.4595836111,138475,"SOMA, DEBORAH",21/06/1983,1983-06-21,138475,Ibirapuera,PC1P7BHG_45123.4595836111,2024-1350,Ibirapuera,D2024.08.20_S03735_I3166_P-7,2024-08-26,1.7,ADMIN,KIDScoreD5 v3,None,AC7,Avoid,None,2024-08-20 12:45:00,3166,2,7,7,Ibirapuera,None,NaN,None,None,None,NaN,None,None,None,NaN,None,None,None,NaN,None,None,None,NaN,None,None,None,NaN,None,None,FRAG2CAT,23.4,2024.08.21 12:11:35,

In [76]:
embryoscope_embrioes_data.loc[embryoscope_embrioes_data['embryo_FertilizationTime'].isin(['2024-01-15 12:45:00'])].sort_values(by='embryo_embryo_number')

C:\Users\FilipeFurlanBellotti\AppData\Local\Temp\ipykernel_23592\2666312698.py:1: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  embryoscope_embrioes_data.loc[embryoscope_embrioes_data['embryo_FertilizationTime'].isin(['2024-01-15 12:45:00'])].sort_values(by='embryo_embryo_number')


,patient_PatientIDx,patient_PatientID,patient_FirstName,patient_LastName,patient_DateOfBirth,prontuario,patient_unit_huntington,treatment_PatientIDx,treatment_TreatmentName,treatment_unit_huntington,embryo_EmbryoID,embryo_KIDDate,embryo_KIDScore,embryo_KIDUser,embryo_KIDVersion,embryo_Description,embryo_EmbryoDescriptionID,embryo_EmbryoFate,embryo_FertilizationMethod,embryo_FertilizationTime,embryo_InstrumentNumber,embryo_Position,embryo_WellNumber,embryo_embryo_number,embryo_unit_huntington,embryo_Name_BlastExpandLast,embryo_Time_BlastExpandLast,embryo_Timestamp_BlastExpandLast,embryo_Value_BlastExpandLast,embryo_Name_BlastomereSize,embryo_Time_BlastomereSize,embryo_Timestamp_BlastomereSize,embryo_Value_BlastomereSize,embryo_Name_EVEN2,embryo_Time_EVEN2,embryo_Timestamp_EVEN2,embryo_Value_EVEN2,embryo_Name_EVEN4,embryo_Time_EVEN4,embryo_Timestamp_EVEN4,embryo_Value_EVEN4,embryo_Name_EVEN8,embryo_Time_EVEN8,embryo_Timestamp_EVEN8,embryo_Value_EVEN8,embryo_Name_FRAG2,embryo_Time_FRAG2,embryo_Timestamp_FRAG2,embryo_Value_FRAG2,embryo_Name_FRAG2CAT,embryo_Time_FRAG2CAT,embryo_Timestamp_FRAG2CAT,embryo_Value_FRAG2CAT,embryo_Name_FRAG4,embryo_Time_FRAG4,embryo_Timestamp_FRAG4,embryo_Value_FRAG4,embryo_Name_FRAG8,embryo_Time_FRAG8,embryo_Timestamp_FRAG8,embryo_Value_FRAG8,embryo_Name_Fragmentation,embryo_Time_Fragmentation,embryo_Timestamp_Fragmentation,embryo_Value_Fragmentation,embryo_Name_ICM,embryo_Time_ICM,embryo_Timestamp_ICM,embryo_Value_ICM,embryo_Name_Line,embryo_Time_Line,embryo_Timestamp_Line,embryo_Value_Line,embryo_Name_MN2Type,embryo_Time_MN2Type,embryo_Timestamp_MN2Type,embryo_Value_MN2Type,embryo_Name_MorphologicalGrade,embryo_Time_MorphologicalGrade,embryo_Timestamp_MorphologicalGrade,embryo_Value_MorphologicalGrade,embryo_Name_MultiNucleation,embryo_Time_MultiNucleation,embryo_Timestamp_MultiNucleation,embryo_Value_MultiNucleation,embryo_Name_Nuclei2,embryo_Time_Nuclei2,embryo_Timestamp_Nuclei2,embryo_Value_Nuclei2,embryo_Name_PN,embryo_Time_PN,embryo_Timestamp_PN,embryo_Value_PN,embryo_Name_Pulsing,embryo_Time_Pulsing,embryo_Timestamp_Pulsing,embryo_Value_Pulsing,embryo_Name_ReexpansionCount,embryo_Time_ReexpansionCount,embryo_Timestamp_ReexpansionCount,embryo_Value_ReexpansionCount,embryo_Name_Strings,embryo_Time_Strings,embryo_Timestamp_Strings,embryo_Value_Strings,embryo_Name_TE,embryo_Time_TE,embryo_Timestamp_TE,embryo_Value_TE,embryo_Name_t2,embryo_Time_t2,embryo_Timestamp_t2,embryo_Value_t2,embryo_Name_t3,embryo_Time_t3,embryo_Timestamp_t3,embryo_Value_t3,embryo_Name_t4,embryo_Time_t4,embryo_Timestamp_t4,embryo_Value_t4,embryo_Name_t5,embryo_Time_t5,embryo_Timestamp_t5,embryo_Value_t5,embryo_Name_t6,embryo_Time_t6,embryo_Timestamp_t6,embryo_Value_t6,embryo_Name_t7,embryo_Time_t7,embryo_Timestamp_t7,embryo_Value_t7,embryo_Name_t8,embryo_Time_t8,embryo_Timestamp_t8,embryo_Value_t8,embryo_Name_t9,embryo_Time_t9,embryo_Timestamp_t9,embryo_Value_t9,embryo_Name_tB,embryo_Time_tB,embryo_Timestamp_tB,embryo_Value_tB,embryo_Name_tEB,embryo_Time_tEB,embryo_Timestamp_tEB,embryo_Value_tEB,embryo_Name_tHB,embryo_Time_tHB,embryo_Timestamp_tHB,embryo_Value_tHB,embryo_Name_tM,embryo_Time_tM,embryo_Timestamp_tM,embryo_Value_tM,embryo_Name_tPB2,embryo_Time_tPB2,embryo_Timestamp_tPB2,embryo_Value_tPB2,embryo_Name_tPNa,embryo_Time_tPNa,embryo_Timestamp_tPNa,embryo_Value_tPNa,embryo_Name_tPNf,embryo_Time_tPNf,embryo_Timestamp_tPNf,embryo_Value_tPNf,embryo_Name_tSB,embryo_Time_tSB,embryo_Timestamp_tSB,embryo_Value_tSB,embryo_Name_tSC,embryo_Time_tSC,embryo_Timestamp_tSC,embryo_Value_tSC,embryo_PatientIDx,embryo_TreatmentName


### 9. Embryoscope Clinisys Combined


In [77]:
print("=== EMBRYOSCOPE CLINISYS COMBINED DATA ===")
combined_data = results['embryoscope_clinisys_combined']
print(f"Records: {len(combined_data)}")
if not combined_data.empty:
    display(combined_data.head(10))
else:
    print("No embryoscope clinisys combined data found.")


=== EMBRYOSCOPE CLINISYS COMBINED DATA ===
Records: 38


,oocito_id,oocito_id_micromanipulacao,oocito_diaseguinte,oocito_Maturidade,oocito_Embriologista,oocito_PI,oocito_TCD,oocito_PGD,oocito_ResultadoPGD,oocito_IdentificacaoPGD,oocito_Fertilizacao,oocito_CorpusculoPolar,oocito_ComentariosDepois,oocito_OocitoDoado,oocito_OrigemOocito,oocito_InseminacaoOocito,oocito_NClivou_D3,oocito_NCelulas_D3,oocito_Frag_D3,oocito_Blastomero_D3,oocito_NClivou_D4,oocito_NCelulas_D4,oocito_Compactando_D4,oocito_MassaInterna_D4,oocito_Trofoblasto_D4,oocito_NClivou_D5,oocito_NCelulas_D5,oocito_Compactando_D5,oocito_MassaInterna_D5,oocito_Trofoblasto_D5,oocito_NClivou_D6,oocito_NCelulas_D6,oocito_Compactando_D6,oocito_MassaInterna_D6,oocito_Trofoblasto_D6,oocito_NClivou_D7,oocito_NCelulas_D7,oocito_Compactando_D7,oocito_MassaInterna_D7,oocito_Trofoblasto_D7,oocito_score_maia,oocito_relatorio_ia,oocito_flag_embryoscope,oocito_embryo_number,micro_codigo_ficha,micro_numero_caso,micro_prontuario,micro_IdadeEsposa_DG,micro_IdadeMarido_DG,micro_Data_DL,micro_horario_inicial_fert,micro_horario_final_fert,micro_Aspiracao_DL,micro_medico_id,micro_responsavel_puncao_id,micro_recepcao_ovulos,micro_Denud_DL,micro_checagem_fertilizacao,micro_checagem_d3,micro_checagem_d5,micro_checagem_d6,micro_checagem_d7,micro_foliculos,micro_oocitos,micro_ICSIDescongelados,micro_pgd_oocito,micro_pgd_d5,micro_data_procedimento,micro_tecnica_injetados,micro_motilidade,micro_morfologia_injetados,micro_gradiente_descontinuo,micro_unidade_id,micro_responsavel_denud_id,micro_profissional_fertilizacao_id,micro_profissional_d3_id,micro_profissional_d5_id,micro_profissional_d6_id,micro_profissional_d7_id,micro_materiais_insumos,micro_local_coleta,micro_viscosidade,micro_volume_amostra_utilizada,micro_morfologia_pre,micro_morfologia_pos,micro_zymot,micro_observacoes_resumo_ciclos,micro_agulha_icsi,micro_lote_agulha_icsi,micro_validade_agulha_icsi,micro_agulha_aspir,micro_lote_agulha_aspir,micro_validade_agulha_aspir,micro_incubadora,micro_abstinencia_dias,micro_meios,micro_responsavel_oocitos_id,micro_fonte,micro_inicio_ps,micro_termino_ps,micro_volume_ps1,micro_CongelamentoOvulos,micro_concentracao_progressivos_pre,micro_conc_ps1,micro_EstadoSptz,micro_check_gradiente_ps,micro_check_swimup,micro_volume_ps2,micro_concentracao_progressivos_pos,micro_conc_ps2,micro_comentarios_ps,micro_responsavel_ps,micro_timelapse,micro_transferidos,micro_Transfer_D5,micro_medico_responsavel_id,micro_azoospermia,micro_Observacoes,micro_responsavel_labfiv,micro_responsavel_labfiv_data,micro_tipo_paciente,micro_alteracoes_oocitarias_normal,micro_alteracoes_oocitarias_granulacao,micro_alteracoes_oocitarias_pelucida,micro_alteracoes_oocitarias_oval,micro_alteracoes_oocitarias_inclusoes,micro_alteracoes_oocitarias_pervitelineo_aumentado,micro_alteracoes_oocitarias_irregular,micro_alteracoes_oocitarias_necrotica,micro_alteracoes_oocitarias_pervitelineo_granular,micro_alteracoes_oocitarias_ser,micro_alteracoes_oocitarias_corpusculo_frag,micro_alteracoes_oocitarias_vacuolos,micro_alteracoes_oocitarias_corpusculo_aum,micro_alteracoes_oocitarias_membrana,micro_alteracoes_oocitarias_corpusculo_peq,micro_alteracoes_oocitarias_citoplasma,micro_alteracoes_oocitarias_corpusculo_deg,micro_controle_anual,micro_maia,cong_em_id,cong_em_CodCongelamento,cong_em_Unidade,cong_em_prontuario,cong_em_paciente,cong_em_Data,cong_em_Hora,cong_em_Ciclo,cong_em_CicloRecongelamento,cong_em_NEmbrioes,cong_em_NPailletes,cong_em_Identificacao,cong_em_Tambor,cong_em_Cane,cong_em_Cane2,cong_em_Tecnica,cong_em_D5,cong_em_D6,cong_em_Observacoes,cong_em_BiologoFIV,cong_em_BiologoFIV2,cong_em_status_financeiro,descong_em_id,descong_em_CodDescongelamento,descong_em_Unidade,descong_em_prontuario,descong_em_doadora,descong_em_DataCongelamento,descong_em_DataDescongelamento,descong_em_Ciclo,descong_em_Identificacao,descong_em_CodCongelamento,descong_em_Tambor,descong_em_Cane,descong_em_PailletesDescongeladas,descong_em_Tecnica,descong_em_DataTransferencia,descong_em_Incubadora,descong_em_transferido

In [78]:
colunas = [
    'oocito_id','oocito_id_micromanipulacao', 'oocito_embryo_number', 'oocito_TCD', 
    'micro_codigo_ficha', 'micro_numero_caso', 'micro_Data_DL', 'micro_data_procedimento', 
    'cong_em_id', 'cong_em_CodCongelamento', 'cong_em_Ciclo', 'cong_em_NEmbrioes', 
    'treatment_TreatmentName', 'embryo_EmbryoID', 'embryo_KIDScore', 'embryo_Description', 'embryo_EmbryoDescriptionID', 'embryo_embryo_number',
    'emb_cong_transferidos', 'emb_cong_id', 'emb_cong_id_oocito', 'emb_cong_id_congelamento', 'emb_cong_id_descongelamento','emb_cong_embriao', 
    'descong_em_id', 'descong_em_transferidos_transferencia'	, 'descong_em_CodDescongelamento', 'descong_em_Ciclo' , 'descong_em_DataDescongelamento', 'descong_em_DataTransferencia', 
    'trat_id', 'trat_data_procedimento', 'trat_data_transferencia', 'trat_resultado_tratamento', 'trat_tentativa'
]

# combined_data.loc[combined_data['oocito_id_micromanipulacao'].isin([21961]), colunas]
# combined_data.loc[combined_data['embryo_EmbryoDescriptionID'].isin(['AA5']), colunas] 
combined_data.loc[:, colunas]

,oocito_id,oocito_id_micromanipulacao,oocito_embryo_number,oocito_TCD,micro_codigo_ficha,micro_numero_caso,micro_Data_DL,micro_data_procedimento,cong_em_id,cong_em_CodCongelamento,cong_em_Ciclo,cong_em_NEmbrioes,treatment_TreatmentName,embryo_EmbryoID,embryo_KIDScore,embryo_Description,embryo_EmbryoDescriptionID,embryo_embryo_number,emb_cong_transferidos,emb_cong_id,emb_cong_id_oocito,emb_cong_id_congelamento,emb_cong_id_descongelamento,emb_cong_embriao,descong_em_id,descong_em_transferidos_transferencia,descong_em_CodDescongelamento,descong_em_Ciclo,descong_em_DataDescongelamento,descong_em_DataTransferencia,trat_id,trat_data_procedimento,trat_data_transferencia,trat_resultado_tratamento,trat_tentativa
0,212875,21961,1,Criopreservado,21961,2764/24,2024-08-20,2024-08-20,26420,E2183/24,2764/24,9,2024-1350,D2024.08.20_S03735_I3166_P-1,9.2,"Euploid, XX",AC1,1,Transferido,76266,212875,26420,16300,#1,16300,1,2441/24,2764/24,2024-11-10,2024-11-10,28530,2024-08-20,NaT,No transfer,4
1,212875,21961,1,Criopreservado,21961,2764/24,2024-08-20,2024-08-20,26420,E2183/24,2764/24,9,2024-1350,D2024.08.20_S03735_I3166_P-1,9.2,"Euploid, XX",AC1,1,Transferido,76266,212875,26420,16300,#1,16300,1,2441/24,2764/24,2024-11-10,2024-11-10,30064,2024-11-10,2024-11-10,Gestação Clínica,5
2,212876,21961,2,Descartado,21961,2764/24,2024-08-20,2024-08-20,<NA>,None,None,<NA>,2024-1350,D2024.08.20_S03735_I3166_P-2,NA,None,AC2,2,None,<NA>,<NA>,<NA>,<NA>,None,<NA>,None,None,None,NaT,NaT,28530,2024-08-20,NaT,No transfer,4
3,212877,21961,3,Criopreservado,21961,2764/24,2024-08-20,2024-08-20,26420,E2183/24,2764/24,9,2024-1350,D2024.08.20_S03735_I3166_P-3,8.9,"Euploid, XX",AC3,3,None,76267,212877,26420,0,#3,<NA>,None,None,None,NaT,NaT,28530,2024-08-20,NaT,No transfer,4
4,212878,21961,4,Criopreservado,21961,2764/24,2024-08-20,2024-08-20,26420,E2183/24,2764/24,9,2024-1350,D2024.08.20_S03735_I3166_P-4,9.4,"Euploid, XX",AC4,4,None,76268,212878,26420,0,#4,<NA>,None,None,None,NaT,NaT,28530,2024-08-20,NaT,No transfer,4
5,212879,21961,5,Descartado,21961,2764/24,2024-08-20,2024-08-20,<NA>,None,None,<NA>,2024-1350,D2024.08.20_S03735_I3166_P-5,1.5,None,AC5,5,None,<NA>,<NA>,<NA>,<NA>,None,<NA>,None,None,None,NaT,NaT,28530,2024-08-20,NaT,No transfer,4
6,212880,21961,6,Descartado,21961,2764/24,2024-08-20,2024-08-20,<NA>,None,None,<NA>,2024-1350,D2024.08.20_S03735_I3166_P-6,NA,None,AC6,6,None,<NA>,<NA>,<NA>,<NA>,None,<NA>,None,None,None,NaT,NaT,28530,2024-08-20,NaT,No transfer,4
7,212881,21961,7,Descartado,21961,2764/24,2024-08-20,2024-08-20,<NA>,None,None,<NA>,2024-1350,D2024.08.20_S03735_I3166_P-7,1.7,None,AC7,7,None,<NA>,<NA>,<NA>,<NA>,None,<NA>,None,None,None,NaT,NaT,28530,2024-08-20,NaT,No transfer,4
8,212882,21961,8,Criopreservado,21961,2764/24,2024-08-20,2024-08-20,26420,E2183/24,2764/24,9,2024-1350,D2024.08.20_S03735_I3166_P-8,3.5,S/ANALISE,AC8,8,None,76269,212882,26420,0,#8,<NA>,None,None,None,NaT,NaT,28530,2024-08-20,NaT,No transfer,4
9,212883,21961,9,Criopreservado,21961,2764/24,2024-08-20,2024-08-20,26420,E2183/24,2764/24,9,2024-1350,D2024.08.20_S03735_I3166_P-9,6.3,S/ANALISE,AC9,9,None,76270,212883,26420,0,#9,<NA>,None,None,None,NaT,NaT,28530,2024-08-20,NaT,No transfer,4


### 11. Planilha Embriologia


In [79]:
print("=== PLANILHA EMBRIOLOGIA DATA ===")
planilha_data = results['planilha_embriologia']
print(f"Records: {len(planilha_data)}")
if not planilha_data.empty:
    display(planilha_data)
else:
    print("No planilha embriologia data found.")


=== PLANILHA EMBRIOLOGIA DATA ===
Records: 2


,PIN,Aux,RESP PREENCHIMENTO,UNIDADE,TIPO DE TRATAMENTO,TIPO DE FET,TIPO BIÓPSIA,TIPO DA DOAÇÃO,DATA INICIO PREPARO ENDOMETRIAL,DATA DA FET,MÊS,NOME DA PACIENTE,STATUS DO RELACIONAMENTO,FONTE PAGADORA,MÉDICO,GRUPO DE IDADE,IDADE MULHER,IDADE DO CONG DE EMBRIÃO,Data do congelamento,ALTURA,PESO,DATA DE NASC.,IDADE ESPERMATOZOIDE,FATOR 1,FATOR 2,PREPARO PARA TRANSFERENCIA,VIA FASE FOLICULAR,VIA LUTHEAL FASE,END. ESPESSURA,ESTAGIO BIOPSIA,Nº BIOPSIADOS,QTD ANALISADOS,QTD FALHA LEITURA,QTD NORMAIS,RESP CONGELAMENTO,QTD EMBRIO PREVIAMENTE CONGELADOS,DIA CRYO,ESTAGIO DESENVOLVIMENTO,DATA CRIO,MÉTODO DE CRIOPRESERVAÇÃO,RESP. DESCONGELAMENTO,Nº DESCONG,Nº SOBREVIVERAM,ESTAGIO DESENVOLVIMENTO3,Nº EMBR RECONGELADOS,Nº TOTAL EMBR. PERMANECEM CONG,RESP. TRANSFERENCIA,HOUVE TRANSFERENCIA?,Embryoglue,Qualidade Embrião transferido,"NÚMERO DA TRANSFER (1ª, 2ª, 3ª...)",DIA ET,ESTAGIO ET,Nº ET,MÉDICO ET,CATÉTER,ASSISTED HATCHING,INTERCORRÊNCIAS,RESULT,TIPO DO RESULTADO,OBS,GRAVIDEZ BIOQUIMICA,GRAVIDEZ CLINICA,DATA 1º US,SG 1º US,Qtd BCF,DATA 2º US,SG 2º US,Qtd BCF2,BCF,EVOLUÇÃO,DATA ABORTO,DATA DIAGNOSTICO GRAV. ECTOPICA,TIPO DE TRATAMENTO GRAV. ECTOP.,DATA INTERRUPÇÃO GRAV. ECTOP.,ESTUDO CITOGENÉTICO,RESULTADO DO ESTUDO GENÉTICO,ID GESTACIONAL,DATA PARTO,TIPO DE PARTO,Nº NASCIDOS,VIABILIDADE BEBÊ 1,PESO 1,ANOMALIA CONGÊNITA,ESTUDO CITOGEN,VIABILIDADE BEBÊ 2,PESO 2,ANOMALIA CONGÊN,ESTUDO CITOGE,VIABILIDADE BEBÊ 3,PESO 3,ANOMALIA CONG,ESTUDO CITOG,VIABILIDADE BEBÊ 4,PESO 4,ANOMALIA CONG.,ESTUDO CITOG.,OHSS,HEMORRAGIA,INFECÇÃO,Coluna1,ALTURA (cm),Auxiliar,Coluna12,IDADE DA MULHER,IDADE DO\n ESPERMATOZOIDE,Qtd BCF_1,line_number,extraction_timestamp,file_name
0,138475,None,LAURA BELOMO,IBIRAPUERA,FET FROM PGT,CYCLE CONTINUATION,PGT-A,None,2024-10-22,2024-11-10,NOVEMBRO,DEBORAH SOMA,None,None,Thais Domingues,4,41,41,None,160.0,60.0,1983-06-21,<NA>,OTHER,None,HORMONAL REPLACEMENT CYCLE,ORAL,COMBINED,10.6,None,<NA>,<NA>,<NA>,<NA>,Bruna Lourenço,7,5000000.0,BLASTOCYST,2024-08-25,VITRIFICATION,Beatriz Aiello,1,1,BLASTOCYST,<NA>,<NA>,Dayane Reis,1,None,None,4,5,BLASTOCYST,1,Thais Domingues,SYDNEY,0,None,EMBRYO TRANSFER,POSITIVO,None,1,1,2024-11-09,1,1,2024-12-23,1,1,POSITIVO,<NA>,NaT,NaT,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,None,<NA>,None,<NA>,739,20251121_155046,CASOS 2024 IBIRA.xlsx
1,138475,None,LAURA BELOMO,IBIRAPUERA,FET FROM PGT,THAWING OF EMBRYO,PGT-A,None,2024-06-07,2024-06-27,JUNHO,DEBORAH SOMA,None,None,Thais Domingues,4,41,40,None,160.0,60.0,1983-06-21,<NA>,NOT AVAILABLE,None,HORMONAL REPLACEMENT CYCLE,ORAL,COMBINED,11.3,None,<NA>,<NA>,<NA>,<NA>,Izadora Reis,1,5.0,BLASTOCYST,2024-02-21,VITRIFICATION,Beatriz Aiello,1,1,BLASTOCYST,<NA>,<NA>,Laura Belomo,1,None,None,2,5,BLASTOCYST,1,Thais Domingues,SYDNEY,0,None,EMBRYO TRANSFER,NÃO ENGRAVIDOU,None,0,0,NaT,<NA>,<NA>,NaT,<NA>,None,None,<NA>,NaT,NaT,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,None,<NA>,None,<NA>,399,20251121_155046,CASOS 2024 IBIRA.xlsx


### 12. Congelamentos Ovulos


In [80]:
print("=== CONGELAMENTOS OVULOS DATA ===")
congelamentos_ovulos_data = results['congelamentos_ovulos']
print(f"Records: {len(congelamentos_ovulos_data)}")
if not congelamentos_ovulos_data.empty:
    display(congelamentos_ovulos_data)
else:
    print("No congelamentos ovulos data found.")


=== CONGELAMENTOS OVULOS DATA ===
Records: 1


,id,CodCongelamento,Unidade,prontuario,paciente,Data,Hora,Ciclo,NOvulos,NPailletes,Identificacao,Tambor,Cane,Cane2,Tecnica,Motivo,BiologoFIV,BiologoFIV2,status_financeiro,hash,extraction_timestamp
0,9436,EGG1569,1,138475,esposa,2015-10-14,1900-01-01 12:00:00,None,22,6,EGG1569,S,2,None,Vitrificação,Preservação da Fertilidade,Mariana Nicolielo,None,None,b0c1e3f2d5fdb192442a7bb9ee508f0f,2025-07-21 21:38:49


### 13. Descongelamentos Ovulos


In [81]:
print("=== DESCONGELAMENTOS OVULOS DATA ===")
descongelamentos_ovulos_data = results['descongelamentos_ovulos']
print(f"Records: {len(descongelamentos_ovulos_data)}")
if not descongelamentos_ovulos_data.empty:
    display(descongelamentos_ovulos_data)
else:
    print("No descongelamentos ovulos data found.")


=== DESCONGELAMENTOS OVULOS DATA ===
Records: 2


,id,CodDescongelamento,Unidade,prontuario,doadora,DataCongelamento,DataDescongelamento,Ciclo,Identificacao,CodCongelamento,Tambor,Cane,PailletesDescongeladas,Tecnica,Observacoes,BiologoFIV,BiologoFIV2,hash,extraction_timestamp
0,3658,DO95/24,1,138475,0,2015-10-14,2024-02-16,-,EGG1569,EGG1569,S,2,3,Desvitrificação,None,Izadora Reis,Bruna Arroio,4d1db3f338041395cb8e086cab37f7f0,2025-07-21 21:38:53
1,3133,DO188/23,1,138475,0,2015-10-14,2023-07-16,-,EGG1569,EGG1569,S,2,03,Desvitrificação,None,Beatriz Aiello,Laura Belomo,c29bfbbf71fc030f9bb4da44f3fc04aa,2025-07-21 21:38:53


### 14. Ovulos Congelados


In [82]:
print("=== OVULOS CONGELADOS DATA ===")
ovulos_congelados_data = results['ovulos_congelados']
print(f"Records: {len(ovulos_congelados_data)}")
if not ovulos_congelados_data.empty:
    display(ovulos_congelados_data)
else:
    print("No ovulos congelados data found.")


=== OVULOS CONGELADOS DATA ===
Records: 22


,id,id_congelamento,id_descongelamento,prontuario,pailletes,ovulo,comentarios,dia2,maturidade,dia2_2,tanque_amostra,caneca_amostra,rack_amostra,hash,extraction_timestamp
0,56207,9436,3658,138475,6,22.0,None,DEG,MII,None,None,None,None,f58140ed2609415182d5bca1fab30d64,2025-10-14 21:04:28
1,56206,9436,3658,138475,6,21.0,None,DEG,MII,None,None,None,None,130544cf96f4d7ca53cb852ccaa5e39b,2025-10-14 21:04:28
2,56205,9436,3658,138475,6,20.0,None,MII,MII,None,None,None,None,17c70891f74015efdfe50b9e3355ed6c,2025-10-14 21:04:28
3,56204,9436,3133,138475,5,19.0,None,DEG,MII,None,None,None,None,5b7c1c0c3b980e2c571c2f84b16b04cc,2025-10-14 21:04:28
4,56203,9436,3133,138475,5,18.0,None,DEG,MII,None,None,None,None,00e23a30790a99912c2cf18655c3b93c,2025-10-14 21:04:28
5,56202,9436,3133,138475,5,17.0,None,MII,MII,None,None,None,None,4301d0c634f166e72d1faa56a37a592a,2025-10-14 21:04:28
6,56201,9436,3658,138475,4,16.0,None,MII,MII,None,None,None,None,8fe2c9a23a654a18828c80b0bd752900,2025-10-14 21:04:28
7,56200,9436,3658,138475,4,15.0,None,MII,MII,None,None,None,None,da70753f147ca23c01c36d80693c0b8b,2025-10-14 21:04:28
8,56199,9436,3658,138475,4,14.0,None,MII,MII,None,None,None,None,37819deef71146c42e61cc67709a691d,2025-10-14 21:04:28
9,56198,9436,3658,138475,4,13.0,None,MII,MII,None,None,None,None,5da390a715fa468f7144c718f3bc667f,2025-10-14 21:04:28


### 12. Planilha Embryoscope Combined


In [83]:
print("=== PLANILHA EMBRYOSCOPE COMBINED DATA ===")
combined_planilha_data = results['planilha_embryoscope_combined']
print(f"Records: {len(combined_planilha_data)}")
if not combined_planilha_data.empty:
    display(combined_planilha_data)
else:
    print("No planilha embryoscope combined data found.")


=== PLANILHA EMBRYOSCOPE COMBINED DATA ===
Records: 37


,oocito_id,oocito_id_micromanipulacao,oocito_diaseguinte,oocito_Maturidade,oocito_RC,oocito_Embriologista,oocito_PI,oocito_TCD,oocito_PGD,oocito_ResultadoPGD,oocito_IdentificacaoPGD,oocito_Fertilizacao,oocito_CorpusculoPolar,oocito_ComentariosDepois,oocito_OocitoDoado,oocito_OrigemOocito,oocito_InseminacaoOocito,oocito_NClivou_D3,oocito_NCelulas_D3,oocito_Frag_D3,oocito_Blastomero_D3,oocito_NClivou_D5,oocito_Compactando_D5,oocito_MassaInterna_D5,oocito_Trofoblasto_D5,oocito_NClivou_D6,oocito_Compactando_D6,oocito_flag_embryoscope,oocito_embryo_number,micro_codigo_ficha,micro_numero_caso,micro_prontuario,micro_IdadeEsposa_DG,micro_IdadeMarido_DG,micro_Data_DL,micro_horario_inicial_fert,micro_horario_final_fert,micro_Aspiracao_DL,micro_medico_id,micro_responsavel_puncao_id,micro_recepcao_ovulos,micro_Denud_DL,micro_checagem_fertilizacao,micro_checagem_d3,micro_checagem_d5,micro_checagem_d6,micro_checagem_d7,micro_foliculos,micro_oocitos,micro_ICSIDescongelados,micro_pgd_oocito,micro_pgd_d5,micro_data_procedimento,micro_tecnica_injetados,micro_motilidade,micro_morfologia_injetados,micro_gradiente_descontinuo,micro_unidade_id,micro_responsavel_denud_id,micro_profissional_fertilizacao_id,micro_profissional_d3_id,micro_profissional_d5_id,micro_profissional_d6_id,micro_profissional_d7_id,micro_materiais_insumos,micro_local_coleta,micro_viscosidade,micro_volume_amostra_utilizada,micro_morfologia_pre,micro_morfologia_pos,micro_zymot,micro_observacoes_resumo_ciclos,micro_agulha_icsi,micro_lote_agulha_icsi,micro_validade_agulha_icsi,micro_agulha_aspir,micro_lote_agulha_aspir,micro_validade_agulha_aspir,micro_incubadora,micro_abstinencia_dias,micro_meios,micro_responsavel_oocitos_id,micro_fonte,micro_inicio_ps,micro_termino_ps,micro_volume_ps1,micro_CongelamentoOvulos,micro_concentracao_progressivos_pre,micro_conc_ps1,micro_EstadoSptz,micro_check_gradiente_ps,micro_check_swimup,micro_volume_ps2,micro_concentracao_progressivos_pos,micro_conc_ps2,micro_comentarios_ps,micro_responsavel_ps,micro_timelapse,micro_transferidos,micro_Transfer_D5,micro_medico_responsavel_id,micro_azoospermia,micro_Observacoes,micro_responsavel_labfiv,micro_responsavel_labfiv_data,micro_tipo_paciente,micro_alteracoes_oocitarias_normal,micro_alteracoes_oocitarias_granulacao,micro_alteracoes_oocitarias_pelucida,micro_alteracoes_oocitarias_oval,micro_alteracoes_oocitarias_inclusoes,micro_alteracoes_oocitarias_pervitelineo_aumentado,micro_alteracoes_oocitarias_irregular,micro_alteracoes_oocitarias_necrotica,micro_alteracoes_oocitarias_pervitelineo_granular,micro_alteracoes_oocitarias_ser,micro_alteracoes_oocitarias_corpusculo_frag,micro_alteracoes_oocitarias_vacuolos,micro_alteracoes_oocitarias_corpusculo_aum,micro_alteracoes_oocitarias_membrana,micro_alteracoes_oocitarias_corpusculo_peq,micro_alteracoes_oocitarias_citoplasma,micro_alteracoes_oocitarias_corpusculo_deg,micro_controle_anual,micro_maia,id_1,cong_em_CodCongelamento,cong_em_Unidade,prontuario_1,cong_em_paciente,cong_em_Data,cong_em_Hora,cong_em_Ciclo,cong_em_CicloRecongelamento,cong_em_NEmbrioes,cong_em_NPailletes,cong_em_Identificacao,cong_em_Tambor,cong_em_Cane,cong_em_Cane2,cong_em_Tecnica,cong_em_D5,cong_em_D6,Observacoes_1,cong_em_BiologoFIV,cong_em_BiologoFIV2,cong_em_status_financeiro,id_2,descong_em_CodDescongelamento,Unidade_1,prontuario_2,descong_em_doadora,descong_em_DataCongelamento,descong_em_DataDescongelamento,Ciclo_1,Identificacao_1,CodCongelamento_1,Tambor_1,Cane_1,descong_em_PailletesDescongeladas,Tecnica_1,descong_em_DataTransferencia,Incubadora_1,descong_em_transferidos_transferencia,descong_em_cateter_transferencia,descong_em_lote_transferencia,descong_em_validade_transferencia,descong_em_sangue_interno_transferencia,descong_em_sangue_externo_transferencia,descong_em_retorno_transferencia,descong_em_responsavel_transferencia,Observacoes_2,BiologoFIV_1,BiologoFIV2_1,id_3,emb_cong_id_oocito,emb_cong_id_congelamento,emb_cong_id_descongelamento,prontuario_3,emb_cong_pailletes,emb_cong_paill